#1. Installing vLLM requirements:
- OS: Linux

- Python: 3.9 – 3.12

- GPU: compute capability 7.0 or higher (e.g., V100, T4, RTX20xx, A100, L4, H100, etc.)

So first, change the Runtime type of the Colab to (maybe) T4-GPU.

*NOTE: maybe you need to attend to gcsfs and fsspec version, just install the gcsfs==2024.9.0

In [ ]:
!pip install gcsfs==2024.9.0

!pip install triton

!pip install vllm

Reason for being yanked: requirements incorrect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: gcsfs
    Found existing installation: gcsfs 2024.10.0
    Uninstalling gcsfs-2024.10.0:
      Successfully uninstalled gcsfs-2024.10.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.0 MB/s eta 0:00:

#2. Serve the Qwen2.5-1.5B Model on vLLM
- run this cell with Linux's [nohup](https://www.ibm.com/docs/en/aix/7.1?topic=n-nohup-command) (No Hang Up) command because the serving process will always on during the request/ response time.
- set datatype to float16 because of the calculation abilities of T4 GPU
- the --api_key default is "EMPTY", you can customize your own.

In [ ]:
!nohup vllm serve Qwen/Qwen2.5-1.5B-Instruct --dtype float16 &

nohup: appending output to 'nohup.out'


# 3. Test API to your served model with OpenAI API
- You can skip this step
- Since the aboved cell run in No Hang Out style, you need to follow the logs in the root/content/nohup.out
- Once the serving process is ready to use, the configuration of client must be success.

In [ ]:
!pip install openai==1.55.2
!pip install httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.5/389.5 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0


In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="EMPTY"
)

In [ ]:
# Test the API with the localhost:8000/v1 route.

completion = client.chat.completions.create(
    model="Qwen/Qwen2.5-1.5B-Instruct",
    messages=[
        {
            "role": "user",
            "content": "Uống thuốc kháng sinh thay đổi màu nước tiểu?"
        }
    ]
)

print(completion.choices[0].message.content)

Uống thuốc kháng sinh thay đổi màu nước tiểu có thể xảy ra, nhưng không phải lúc nào cũng như vậy. Điều này phụ thuộc vào nhiều yếu tố và không phải lúc nào cũng là dấu hiệu xấu.

1. Một số loại thuốc kháng sinh, như tetracycline và doxycycline, có thể khiến nước tiểu có màu vàng hoặc nâu. Điều này là do thuốc làm thay đổi cấu trúc của protein trong nước tiểu.

2. Một số thuốc khác, như một số loại rifampicin và rifabutin, có thể khiến nước tiểu có màu vàng hoặc nâu.

3. Một số loại thuốc kháng sinh khác, như amoxicillin và ampicillin, cũng có thể làm thay đổi màu nước tiểu, nhưng thường không như những loại thuốc trên.

Tuy nhiên, nếu bạn đang uống thuốc kháng sinh và nước tiểu của bạn có màu khác thường, có thể bạn đã nhầm lẫn màu nước tiểu với màu thuốc. Nếu bạn còn lo lắng về vấn đề này, hãy liên hệ với bác sĩ để được tư vấn.


#4. Use ngrok to create tunel to make API accessable from your local machine
- First, you need to sign up for a [ngrok](https://ngrok.com/) account and go to the "Your Authtoken" to create and get a authentication token
- Then you need to add your token to Secrets list of this Colab Notebook as the key's name: "ngrok-authtoken"
- After all, you may want to test the API from your local machine with curl commands.

In [ ]:
!pip install pyngrok

from google.colab import userdata
ngr_tk = userdata.get('ngrok-authtoken')

!ngrok config add-authtoken {ngr_tk}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
import os


# Step 1: Open a tunnel to the local server on port 8000
public_url = ngrok.connect(8000)

# Step 2: Print the public URL
print(f"Your ngrok public URL is: {public_url}")

Your ngrok public URL is: NgrokTunnel: "https://e01b-35-197-39-137.ngrok-free.app" -> "http://localhost:8000"
